In [14]:
import pandas as pd
from pyomo.environ import *
from clean_data_cor import *
# from itertools import product
# import random 

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [38]:
# {start} chamar dados
demanda = pd.read_csv('C:/Tesis_wilmer/tesis/instancias/demanda.csv')
preco_ = pd.read_csv('C:/Tesis_wilmer/tesis/instancias/preco.csv')
rota = pd.read_csv('C:/Tesis_wilmer/tesis/instancias/rota.csv')
# {end} chamar dados

rota = [0] + eval(rota['Route'][0])
periodo_lim = demanda['DBD'].tolist()[-10:]
demanda_lim = demanda[demanda['DBD'].isin(periodo_lim)]
origin1, destin, oridest, vagones, periodos, stations, clases, preco, dem = clean_data(demanda_lim, preco_)

origin1 = [i for i in rota if i in origin1]
origin2 = [0] + origin1

destin = [i for i in rota if i in destin]
n = len(rota)-1
cap = 561


c:\Tesis_wilmer\tesis\clean_data_cor.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  records_to_correct['clave_completa'] = records_to_correct['Origin'].astype(str) + '_' + records_to_correct['Destination'].astype(str) + '_' + records_to_correct['Vagon'].astype(str) + '_' + records_to_correct['Class'].astype(str) + '_' + records_to_correct['DBD'].astype(str)


In [39]:
# instanciar o modelo
model = AbstractModel()

# Escalares
model.n = Param(initialize = n)
model.Q = Param(initialize = cap)

# conjuntos
model.I = Set(initialize = origin1)
model.I2 = Set(initialize = origin2)
model.J = Set(initialize = destin)
model.OD = Set(dimen=2, initialize = oridest)
model.V = Set(initialize = vagones)
model.T = Set(initialize = periodos)
model.VK = Set(model.V, initialize=clases)
def wagons_class(model):
    return ((v,k) for v in clases.keys() for k in clases[v])
model.VKI = Set(dimen=2,initialize=wagons_class)

#parâmetros indexados
model.P = Param(model.OD, model.VKI, initialize = preco)
model.d = Param(model.OD, model.VKI, model.T, initialize = dem)

# variáveis de decisão
model.X = Var(model.I2, model.J, model.VKI, model.T, within = NonNegativeIntegers)
model.Y = Var(model.I2, model.J, model.VKI, model.T, within = NonNegativeIntegers)
# model.AU = Var(within = NonNegativeIntegers)

# função objetivo
def fo1(model):
    return sum(model.P[i,j,v,k]*(model.X[i,j,v,k,t]) for i,j in model.OD for v,k in model.VKI for t in model.T)
model.OBJ1 = Objective(rule = fo1, sense = maximize)

# restrições
def rest1(model , j):
    # if rota.index(j) > 1:
    return sum((model.X[i,j,v,k,t]) for i,_ in model.OD for v,k in model.VKI for t in model.T if rota.index(i)<rota.index(j)) <= model.Q
    # else:
    #     return Constraint.Skip
model.ConstrainRest1 = Constraint(model.J, rule = rest1)

# def rest2(model , i):
#     if rota.index(i) < model.n:
#         return sum((model.X[i,j,v,k,t]) for _,j in model.OD for v,k in model.VKI for t in model.T if rota.index(j)>rota.index(i) ) <= model.Q
#     else:
#         return Constraint.Skip
# model.ConstrainRest2 = Constraint(model.I, rule = rest2)

def authorizationCap(model):
    return  sum(model.Y[i,j,v,list(model.VK[v])[0],t] for i,j in model.OD for v in model.VK for t in model.T) <= model.Q
model.ConstrainAuthorizationCap = Constraint(rule = authorizationCap)

def authorization(model, i, j, v, k):
    lista = list(model.VK[v])
    pos_k = lista.index(k)
    lista_sort = sorted(lista, reverse=True)
    if pos_k == 0:
        return  sum(model.Y[i,j,v,lista_sort[pos_k],t] for t in model.T) >=  sum(model.X[i,j,v,lista_sort[pos_k],t] for t in model.T )
    elif pos_k <= len(lista)-1 :
        return  sum(model.Y[i,j,v,lista_sort[pos_k],t] for t in model.T) >=  sum(model.Y[i,j,v,lista_sort[pos_k-1],t] for t in model.T) + sum(model.X[i,j,v,lista_sort[pos_k],t] for t in model.T )
    else:
        return Constraint.Skip
model.ConstrainAuthorization = Constraint(model.OD, model.VKI, rule = authorization)

def classe(model, i, j, v, k, t):
    lista = list(model.VK[v])
    if k != lista[-1]:
        pos_k = lista.index(k)
        return model.Y[i,j,v,k,t] >= model.Y[i,j,v,lista[pos_k+1],t]
    else:
        return Constraint.Skip
model.ConstrainClasse = Constraint(model.OD, model.VKI, model.T, rule = classe)

def assignment(model, i, j, v, k, t):
    if rota.index(i) < rota.index(j):
        return model.X[i,j,v,k,t] <= model.d[i,j,v,k,t]
    else:
        return Constraint.Skip
model.ConstrainAssignment = Constraint(model.OD, model.VKI, model.T,rule = assignment)

instance = model.create_instance()
# instance.pprint()

opt = SolverFactory('gurobi')
results = opt.solve(instance, tee=True)
# print(results)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-12
Read LP format model from file C:\Users\LAB_C\AppData\Local\Temp\tmpo_am8a6t.pyomo.lp
Reading time = 0.12 seconds
x1: 75521 rows, 76800 columns, 220160 nonzeros
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i7-13700T, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 75521 rows, 76800 columns and 220160 nonzeros
Model fingerprint: 0x0185d78c
Variable types: 0 continuous, 76800 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 6e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 71455 rows and 71733 columns
Presolve time: 0.11s
Presolved: 4066 rows, 5067 columns, 17077 nonzeros
Found heuristic solution: objective 8951.4000000
Va

In [40]:
print('Valor da função objetivo: ', '\033[1m' + '\033[32m' + str(value(instance.OBJ1)) + '\033[0m')

print('')

# print('Valores das variáveis de decisão: ')
lista = []
for i,j in instance.OD:
    for v,k in instance.VKI:
        for t in instance.T:
            # if value(instance.Y[i,j,v,k,t])!=0 or value(instance.X[i,j,v,k,t])!=0:
                # print(i,j,v,k,t, ' - ',value(instance.Y[i,j,v,k,t]), ' - ', value(instance.X[i,j,v,k,t]) )
            lista.append([str(i) + '-' + str(j),v, k, t, value(instance.P[i,j,v,k]), value(instance.d[i,j,v,k,t]) , value(instance.X[i,j,v,k,t]), value(instance.Y[i,j,v,k,t]) ])

a = pd.DataFrame(lista, columns=['o-d','Vagon','classe','Periodo','preco','demanda','Assignments','Authorizations'])
# a.to_excel('prueba_m2.xlsx', index=False)

Valor da função objetivo:  38277.34



In [41]:
# def authorization(model, i, j, v, k):
#     lista = list(model.VK[v])
#     if rota.index(i) < rota.index(j):
#         pos_k = lista.index(k)
#         sub_list = lista[pos_k:]
#         return  sum(model.Y[i,j,v,k,t] for t in model.T) >= sum(model.X[i,j,v,r,t] for r in sub_list for t in model.T )
#     else:
#         return Constraint.Skip
# model.ConstrainAuthorization = Constraint(model.OD, model.VKI, rule = authorization)


# rota = eval(rota['Route'][0])
# periodo_lim = demanda['DBD'].tolist()[-10:]
# demanda_lim = demanda[demanda['DBD'].isin(periodo_lim)]
# origin1, destin, oridest, vagones, periodos, stations, clases, preco, dem = clean_data(demanda_lim, preco_)
# origin2 = [0] + [i for i in rota if i in stations]
# origin3 = [0] + origin1
# n = len(stations)
# cap = 561

# route = 'C:/Jupyter/TESIS/instancias/real/testInstance_Amtrak.csv'
# start_estation = 'NYP'
# commer_class_hierar = {'Y': '1-1','YA': '1-2','YB': '1-3','YC': '1-4','YD': '1-5','YE': '1-6','YF': '1-7','YG': '1-8','JY': '2-1','JA': '2-2','JB': '2-3','JC': '2-4','JD': '2-5','JE': '2-6','JF': '2-7','JG': '2-8'}
